In [4]:
import os
import random

import numpy as np
import quaternion

import xml.etree.cElementTree as ET
import xml.dom.minidom as minidom

from deap import base
from deap import creator
from deap import gp
from deap import tools
from deap import algorithms

from dm_control import mujoco
from dm_control.suite import base as control_base
from dm_control.suite import common
from dm_control.utils import containers
from dm_control.utils import rewards
from dm_control.rl import control
from dm_control.rl.control import PhysicsError

import matplotlib.pyplot as plt
from IPython.display import clear_output
import collections
from skimage import io, transform

from deap import base, creator, gp, tools
import operator
from genome_synth import Genesis, prettify
from findtarget import FindTarget, Physics

In [5]:
gen = Genesis()
print(gen.mk_leaf())
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(gen.combine_trees, 2)
pset.addTerminal(gen.mk_leaf)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr_init", gp.genFull, pset=pset, min_=2, max_=4)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def get_model_and_assets():
    curpath = os.getcwd()
    return common.read_model(curpath + '/dust.xml'), common.ASSETS

def capture(_savename, individuals):
    imnames = set()
    picidx = 0
    for individual in individuals:
        creation = gp.compile(individual, pset)
        tree = ET.fromstring(prettify(gen.mk_model(creation)))
        tree = ET.ElementTree(tree)
        tree.write('dust.xml')
        _DEFAULT_TIME_LIMIT = 10
        _CONTROL_TIMESTEP = .04

        genesis_physics = Physics.from_xml_string(*get_model_and_assets())
        genesis_physics.set_genesis(gen)
        genesis_task = FindTarget()
        genesis_env = control.Environment(genesis_physics, 
                                         genesis_task,
                                         control_timestep=_CONTROL_TIMESTEP,
                                         time_limit=_DEFAULT_TIME_LIMIT)
        action_spec = genesis_env.action_spec()
        observation_spec = genesis_env.observation_spec()

        time_step = genesis_env.reset()
        curtime = 0.0
        while(not time_step.last()):
            try:
                action = np.ones(action_spec.shape[0]) * np.sin(curtime)
                time_step = genesis_env.step(action)
                savename = "basedgod_{0:04}.jpg".format(picidx)
                picidx += 1
                imnames.add(savename)
                curtime += _CONTROL_TIMESTEP
                io.imsave(savename, genesis_env.physics.render(480, 480, camera_id='tracking_top'))
            except PhysicsError:
                print('except')
                break
    if os.path.isfile(_savename + '.mp4'):
        os.remove(_savename + '.mp4')
    !!ffmpeg -f image2 -pattern_type sequence -i "basedgod_%4d.jpg" -qscale:v 0 {_savename + '.mp4'}
    for name in imnames:
        os.remove(name)
    
def evalGen(individual):
    creation = gp.compile(individual, pset)
    #print(creation)
    tree = ET.fromstring(prettify(gen.mk_model(creation)))
    tree = ET.ElementTree(tree)
    tree.write('dust.xml')
    _DEFAULT_TIME_LIMIT = 10
    _CONTROL_TIMESTEP = .04
    display_stride = 1 / .04 // 24
    picidx = 0
    imnames = set()
    
    genesis_physics = Physics.from_xml_string(*get_model_and_assets())
    genesis_physics.set_genesis(gen)
    genesis_task = FindTarget()
    genesis_env = control.Environment(genesis_physics, 
                                     genesis_task,
                                     control_timestep=_CONTROL_TIMESTEP,
                                     time_limit=_DEFAULT_TIME_LIMIT)
    action_spec = genesis_env.action_spec()
    observation_spec = genesis_env.observation_spec()
    
    
    idx = 0
    
    maxreward = 0.0
    max_energy_penalty = 0.0
    
    time_step = genesis_env.reset()
    curtime = 0.0
    
    while(not time_step.last()):
        try:
            action = np.ones(action_spec.shape[0]) * np.sin(curtime)
            time_step = genesis_env.step(action)
            reward = time_step.reward
            if(maxreward < reward):
                maxreward = reward
            curtime += _CONTROL_TIMESTEP
            idx += 1
        except PhysicsError:
            print('except')
            maxreward -= 500
            break
    return (maxreward,)

toolbox.register("evaluate", evalGen)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=2, max_=4)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)




{'name': '0', 'offset': array([0., 0., 0.]), 'rotation': array([  4.35138327,  73.48245917, 147.89808957]), 'dimensions': [0.11681158550774173, 0.022304276521091338, 0.022304276521091338], 'joint_range': 164.66103466737607, 'children': [], 'priority': 0.11135670551724386}


C:\Users\Hex\Anaconda3\envs\keras_tflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Hex\Anaconda3\envs\keras_tflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [6]:
def main():
    pop = toolbox.population(n=15)
    hof = tools.HallOfFame(5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 5, stats, halloffame=hof)
    return pop, hof, stats

if __name__ == "__main__":
    pop, hoff, _ = main()
    capture("basedgyod", hoff)
    
    

1
12
1
12
13
48
15
48
11
48
6
24
1
12
4
24
4
24
1
12
16
48
15
48
4
24
11
48
15
48
gen	nevals	avg     	std       	min      	max     
0  	15    	0.118443	0.00947235	0.0931907	0.124989
10
30
1
12
1
9
1
15
33
50
1
12
1
9
5
27
1
9
1
15
1  	10    	0.175899	0.19589   	0.120228 	0.908822
10
21
0
6
0
6
1
15
1
9
1
12
1
12
4
15
0
6
1
12
2  	10    	0.229337	0.266516  	0.124018 	0.908822
0
9
1
9
3
12
1
9
1
9
1
9
1
9
1
12
3  	8     	0.380793	0.36362   	0.120027 	0.908822
1
9
0
9
5
24
1
12
0
6
4  	5     	0.698662	0.346267  	0.123969 	0.908822
1
12
0
6
5
30
1
9
1
15
0
9
1
9
5  	7     	0.565596	0.375863  	0.124361 	0.908822
1
9
11
30
0
12
1
12
4
24
